# 0. Code example for Markowitz model
(no shorting allowed, porfolio minimize risk, and rate of return is bigger than r_min)

In [ ]:
from cvxopt import matrix
from cvxopt.solvers import qp
import numpy as np

In [ ]:
n = 4
Cov = matrix([[0.076900, 0.039968, 0.018111, -0.000288 ],
[ 0.039968, 0.050244, 0.019033, -0.000060 ],
[ 0.018111, 0.019033, 0.021381, 0.007511 ],
[-0.000288, -0.000060, 0.007511, 0.008542 ]])
Mean = matrix([0.0073, 0.0346, 0.0444, 0.0271])
r_min = 0.035

In [ ]:
G = matrix(np.concatenate((-np.transpose(Mean), -np.identity(n)), 0))
h = matrix(np.concatenate((-np.ones((1,1))*r_min, np.zeros((n,1))), 0))

In [ ]:
A = matrix(1.0, (1,n))
b = matrix(1.0)
q = matrix(np.zeros((n, 1)))
sol = qp(Cov, q, G, h, A, b)

     pcost       dcost       gap    pres   dres
 0:  9.6979e-03 -9.9747e-01  6e+00  2e+00  3e+00
 1:  9.7719e-03 -6.4156e-01  7e-01  4e-02  4e-02
 2:  9.2936e-03 -2.0327e-01  2e-01  1e-02  1e-02
 3:  8.7597e-03 -5.4022e-02  7e-02  1e-03  1e-03
 4:  8.2687e-03  2.5228e-03  6e-03  1e-04  1e-04
 5:  5.7185e-03  4.1849e-03  2e-03  2e-16  2e-17
 6:  5.3588e-03  5.2547e-03  1e-04  1e-16  4e-17
 7:  5.3134e-03  5.3091e-03  4e-06  7e-18  4e-17
 8:  5.3124e-03  5.3123e-03  5e-08  1e-17  2e-16
Optimal solution found.


In [ ]:
sol

{'x': <4x1 matrix, tc='d'>,
 'y': <1x1 matrix, tc='d'>,
 's': <5x1 matrix, tc='d'>,
 'z': <5x1 matrix, tc='d'>,
 'status': 'optimal',
 'gap': 4.788276286757306e-08,
 'relative gap': 9.01349562149405e-06,
 'primal objective': 0.005312388940918964,
 'dual objective': 0.005312341058156099,
 'primal infeasibility': 9.813077866773595e-18,
 'dual infeasibility': 2.408810521851119e-16,
 'primal slack': 6.653445959311444e-09,
 'dual slack': 1.3051906369130429e-09,
 'iterations': 8}

In [ ]:
print(sol['x'])

[ 8.47e-07]
[ 4.33e-02]
[ 4.38e-01]
[ 5.19e-01]



# 1. Markowitz code using in Korea market & with risk free rate(= KTB, Korea Treasury Bond)

## 1. DATA 전처리(기존 stock data to rate of return으로 변경 + risk free col 추가)

KTB data 출처: https://ecos.bok.or.kr/#/StatisticsByTheme/KoreanStat100

- 한국주식시장을 기준으로하기 Korea Treasury bond의 rate를 사용하는 것이 바람직하다고 생각

- 3년물을 기준으로 36을 나눈값(1달 rate를 구해서 이용)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch import nn, optim, cuda, Tensor
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import time
import copy

In [3]:
!pwd

/content


In [4]:
!pip install chardet

In [5]:
class CustomDataset(Dataset):
    def __init__(self, path):
        self.data = pd.read_csv(path, delimiter=',', encoding='UTF8')
        self.x_data = self.data.iloc[:, 0:-1]
        self.y_data = self.data.iloc[:, [-1]]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.tensor(self.x_data.iloc[idx].values, dtype=torch.float32)
        y = torch.tensor(self.y_data.iloc[idx].values, dtype=torch.float32)
        return x, y

In [6]:
rf_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/MARKET_KOREA/risk_free(2000-2020).csv', delimiter=',')

In [7]:
rf_data

,year,3year_KTP,5year_KTP,10year_KTP,3year_CTP,FF_rate,1month_KTP
0,1999,7.69,8.59,-,8.86,4.75,0.006193
1,2000,8.30,8.67,7.76,9.35,5.25,0.006667
2,2001,5.68,6.21,6.86,7.05,4.00,0.004614
3,2002,5.78,6.26,6.59,6.56,4.25,0.004694
4,2003,4.55,4.76,5.05,5.43,3.75,0.003715
5,2004,4.11,4.35,4.73,4.73,3.25,0.003362
6,2005,4.27,4.52,4.95,4.68,3.75,0.003491
7,2006,4.83,4.96,5.15,5.17,4.50,0.003939
8,2007,5.23,5.28,5.35,5.70,5.00,0.004257
9,2008,5.27,5.36,5.57,7.02,3.00,0.004289


In [8]:
stock_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/dataset/MARKET_KOREA/dataset(2000-2020).xlsx', thousands= ',')

In [9]:
stock_data['Symbol']=stock_data['Symbol'].astype(str)

In [10]:
stock_data

,Symbol,A005930,A000660,A005490,A005380,A006400,A000270,A012330,A035720,A003550,...,A008800,A010050,A015350,A023890,A028040,A031390,A033110,A033430,A036260,A036490
0,Symbol Name,삼성전자,SK하이닉스,POSCO홀딩스,현대차,삼성SDI,기아,현대모비스,카카오,LG,...,행남사,우리종금,부산가스,한국아트라스비엑스,미래SCI,녹십자셀,코너스톤네트웍스,디에스티,이매진아시아,SK머티리얼즈
1,1999-12-28 00:00:00,5320,473074,125000,18000,47200,7100,5750,34624,42531,...,4762,1615,15700,4830,44191,2007335,3690,812543,21328,18800
2,2000-01-31 00:00:00,5580,518222,133000,17100,44500,6720,6230,21769,47389,...,7470,1157,12700,3860,48256,1728974,2800,746062,16506,7050
3,2000-02-29 00:00:00,5120,351370,111500,12050,32900,4470,4070,32966,35482,...,8257,1171,10700,7260,79994,4448870,6750,799986,30015,14500
4,2000-03-31 00:00:00,6700,482889,106500,13400,41000,5000,4070,18364,37437,...,14939,942,13000,6280,77748,3289657,5800,724197,62767,10800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2020-08-31 00:00:00,54000,75100,184500,176500,452500,42450,224000,81692,78124,...,1770,529,50900,49700,279,45300,204,370,2220,246600
250,2020-09-29 00:00:00,58200,84000,196000,178500,433500,46900,230000,73161,70000,...,1770,511,50400,50000,279,40700,204,370,2220,227500
251,2020-10-30 00:00:00,56600,79900,208000,164500,442000,50500,225500,66237,64048,...,1770,513,49550,48700,279,37900,204,370,2220,222100
252,2020-11-30 00:00:00,66700,97500,234000,181500,533000,57800,244000,73864,66693,...,1770,548,50300,55600,279,42650,204,370,2220,259700


In [11]:
stock_data.shape

(254, 665)

In [12]:
stock_data.index

RangeIndex(start=0, stop=254, step=1)

In [13]:
stock_data.columns

Index(['Symbol', 'A005930', 'A000660', 'A005490', 'A005380', 'A006400',
       'A000270', 'A012330', 'A035720', 'A003550',
       ...
       'A008800', 'A010050', 'A015350', 'A023890', 'A028040', 'A031390',
       'A033110', 'A033430', 'A036260', 'A036490'],
      dtype='object', length=665)

new dataframe만들기
- index: 연도-달
- first column: risk free rate
- 2:-1: stock name

In [14]:
rStock_data = pd.DataFrame()

In [15]:
date = stock_data.iloc[2:,0]

In [16]:
date

2      2000-01-31 00:00:00
3      2000-02-29 00:00:00
4      2000-03-31 00:00:00
5      2000-04-28 00:00:00
6      2000-05-31 00:00:00
              ...         
249    2020-08-31 00:00:00
250    2020-09-29 00:00:00
251    2020-10-30 00:00:00
252    2020-11-30 00:00:00
253    2020-12-30 00:00:00
Name: Symbol, Length: 252, dtype: object

In [17]:
rStock_data['date'] = date.map(lambda x:x[0:7])

In [18]:
rStock_data

,date
2,2000-01
3,2000-02
4,2000-03
5,2000-04
6,2000-05
...,...
249,2020-08
250,2020-09
251,2020-10
252,2020-11


In [19]:
rStock_data.reset_index(drop=True)

,date
0,2000-01
1,2000-02
2,2000-03
3,2000-04
4,2000-05
...,...
247,2020-08
248,2020-09
249,2020-10
250,2020-11


In [20]:
def risk_free(date):
    year = date[0:4]
    if year == '2000':
      return rf_data.loc[1]['1month_KTP']
    elif year == '2001':
      return rf_data.loc[2]['1month_KTP']
    elif year == '2002':
      return rf_data.loc[3]['1month_KTP']
    elif year == '2003':
      return rf_data.loc[4]['1month_KTP']
    elif year == '2004':
      return rf_data.loc[5]['1month_KTP']
    elif year == '2005':
      return rf_data.loc[6]['1month_KTP']
    elif year == '2006':
      return rf_data.loc[7]['1month_KTP']
    elif year == '2007':
      return rf_data.loc[8]['1month_KTP']
    elif year == '2008':
      return rf_data.loc[9]['1month_KTP']
    elif year == '2009':
      return rf_data.loc[10]['1month_KTP']
    elif year == '2010':
      return rf_data.loc[11]['1month_KTP']
    elif year == '2011':
      return rf_data.loc[12]['1month_KTP']
    elif year == '2012':
      return rf_data.loc[13]['1month_KTP']
    elif year == '2013':
      return rf_data.loc[14]['1month_KTP']
    elif year == '2014':
      return rf_data.loc[15]['1month_KTP']
    elif year == '2015':
      return rf_data.loc[16]['1month_KTP']
    elif year == '2016':
      return rf_data.loc[17]['1month_KTP']
    elif year == '2017':
      return rf_data.loc[18]['1month_KTP']
    elif year == '2018':
      return rf_data.loc[19]['1month_KTP']
    elif year == '2019':
      return rf_data.loc[20]['1month_KTP']
    elif year == '2020':
      return rf_data.loc[21]['1month_KTP']
    else:
      return 0

In [21]:
rStock_data['risk_free'] = rStock_data['date'].apply(risk_free)

In [22]:
rStock_data

,date,risk_free
2,2000-01,0.006667
3,2000-02,0.006667
4,2000-03,0.006667
5,2000-04,0.006667
6,2000-05,0.006667
...,...,...
249,2020-08,0.000821
250,2020-09,0.000821
251,2020-10,0.000821
252,2020-11,0.000821


In [23]:
df = stock_data

In [24]:
df

,Symbol,A005930,A000660,A005490,A005380,A006400,A000270,A012330,A035720,A003550,...,A008800,A010050,A015350,A023890,A028040,A031390,A033110,A033430,A036260,A036490
0,Symbol Name,삼성전자,SK하이닉스,POSCO홀딩스,현대차,삼성SDI,기아,현대모비스,카카오,LG,...,행남사,우리종금,부산가스,한국아트라스비엑스,미래SCI,녹십자셀,코너스톤네트웍스,디에스티,이매진아시아,SK머티리얼즈
1,1999-12-28 00:00:00,5320,473074,125000,18000,47200,7100,5750,34624,42531,...,4762,1615,15700,4830,44191,2007335,3690,812543,21328,18800
2,2000-01-31 00:00:00,5580,518222,133000,17100,44500,6720,6230,21769,47389,...,7470,1157,12700,3860,48256,1728974,2800,746062,16506,7050
3,2000-02-29 00:00:00,5120,351370,111500,12050,32900,4470,4070,32966,35482,...,8257,1171,10700,7260,79994,4448870,6750,799986,30015,14500
4,2000-03-31 00:00:00,6700,482889,106500,13400,41000,5000,4070,18364,37437,...,14939,942,13000,6280,77748,3289657,5800,724197,62767,10800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2020-08-31 00:00:00,54000,75100,184500,176500,452500,42450,224000,81692,78124,...,1770,529,50900,49700,279,45300,204,370,2220,246600
250,2020-09-29 00:00:00,58200,84000,196000,178500,433500,46900,230000,73161,70000,...,1770,511,50400,50000,279,40700,204,370,2220,227500
251,2020-10-30 00:00:00,56600,79900,208000,164500,442000,50500,225500,66237,64048,...,1770,513,49550,48700,279,37900,204,370,2220,222100
252,2020-11-30 00:00:00,66700,97500,234000,181500,533000,57800,244000,73864,66693,...,1770,548,50300,55600,279,42650,204,370,2220,259700


In [25]:
df.columns=df.iloc[0].to_list()
df.drop([0], axis=0, inplace=True)

In [26]:
df.reset_index(drop=True)

,Symbol Name,삼성전자,SK하이닉스,POSCO홀딩스,현대차,삼성SDI,기아,현대모비스,카카오,LG,...,행남사,우리종금,부산가스,한국아트라스비엑스,미래SCI,녹십자셀,코너스톤네트웍스,디에스티,이매진아시아,SK머티리얼즈
0,1999-12-28 00:00:00,5320,473074,125000,18000,47200,7100,5750,34624,42531,...,4762,1615,15700,4830,44191,2007335,3690,812543,21328,18800
1,2000-01-31 00:00:00,5580,518222,133000,17100,44500,6720,6230,21769,47389,...,7470,1157,12700,3860,48256,1728974,2800,746062,16506,7050
2,2000-02-29 00:00:00,5120,351370,111500,12050,32900,4470,4070,32966,35482,...,8257,1171,10700,7260,79994,4448870,6750,799986,30015,14500
3,2000-03-31 00:00:00,6700,482889,106500,13400,41000,5000,4070,18364,37437,...,14939,942,13000,6280,77748,3289657,5800,724197,62767,10800
4,2000-04-28 00:00:00,6000,345481,86800,10450,46300,5040,4150,9406,30210,...,9364,723,9210,4200,50968,1911134,8150,530517,39441,8600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,2020-08-31 00:00:00,54000,75100,184500,176500,452500,42450,224000,81692,78124,...,1770,529,50900,49700,279,45300,204,370,2220,246600
249,2020-09-29 00:00:00,58200,84000,196000,178500,433500,46900,230000,73161,70000,...,1770,511,50400,50000,279,40700,204,370,2220,227500
250,2020-10-30 00:00:00,56600,79900,208000,164500,442000,50500,225500,66237,64048,...,1770,513,49550,48700,279,37900,204,370,2220,222100
251,2020-11-30 00:00:00,66700,97500,234000,181500,533000,57800,244000,73864,66693,...,1770,548,50300,55600,279,42650,204,370,2220,259700


In [27]:
# 수익율을 계산할 데이터프레임 생성
returns_df = pd.DataFrame(df.iloc[1:,1:])  # 결과를 저장할 데이터프레임 생성
returns_df

,삼성전자,SK하이닉스,POSCO홀딩스,현대차,삼성SDI,기아,현대모비스,카카오,LG,삼성화재,...,행남사,우리종금,부산가스,한국아트라스비엑스,미래SCI,녹십자셀,코너스톤네트웍스,디에스티,이매진아시아,SK머티리얼즈
2,5580,518222,133000,17100,44500,6720,6230,21769,47389,36792,...,7470,1157,12700,3860,48256,1728974,2800,746062,16506,7050
3,5120,351370,111500,12050,32900,4470,4070,32966,35482,30330,...,8257,1171,10700,7260,79994,4448870,6750,799986,30015,14500
4,6700,482889,106500,13400,41000,5000,4070,18364,37437,27100,...,14939,942,13000,6280,77748,3289657,5800,724197,62767,10800
5,6000,345481,86800,10450,46300,5040,4150,9406,30210,22700,...,9364,723,9210,4200,50968,1911134,8150,530517,39441,8600
6,6160,345481,85000,12100,43000,6180,3770,14441,26064,25700,...,6203,720,8000,3440,39738,1519508,14500,454729,45610,8090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,54000,75100,184500,176500,452500,42450,224000,81692,78124,187000,...,1770,529,50900,49700,279,45300,204,370,2220,246600
250,58200,84000,196000,178500,433500,46900,230000,73161,70000,182000,...,1770,511,50400,50000,279,40700,204,370,2220,227500
251,56600,79900,208000,164500,442000,50500,225500,66237,64048,179000,...,1770,513,49550,48700,279,37900,204,370,2220,222100
252,66700,97500,234000,181500,533000,57800,244000,73864,66693,188000,...,1770,548,50300,55600,279,42650,204,370,2220,259700


In [28]:
df=df.iloc[:,1:]

In [29]:
df.columns

Index(['삼성전자', 'SK하이닉스', 'POSCO홀딩스', '현대차', '삼성SDI', '기아', '현대모비스', '카카오',
       'LG', '삼성화재',
       ...
       '행남사', '우리종금', '부산가스', '한국아트라스비엑스', '미래SCI', '녹십자셀', '코너스톤네트웍스', '디에스티',
       '이매진아시아', 'SK머티리얼즈'],
      dtype='object', length=664)

In [30]:
df = df.astype(int)

In [31]:
# 각 주식의 수익율을 계산하여 새로운 데이터프레임에 추가
for column in df.columns:
    current_prices = df[column].values
    previous_prices = df[column].shift(1).values  # 이전 달의 가격
    returns = (current_prices[1:] - previous_prices[1:]) / previous_prices[1:]  # 수익율 계산
    returns_df[column] = returns

# 결과 출력
returns_df

,삼성전자,SK하이닉스,POSCO홀딩스,현대차,삼성SDI,기아,현대모비스,카카오,LG,삼성화재,...,행남사,우리종금,부산가스,한국아트라스비엑스,미래SCI,녹십자셀,코너스톤네트웍스,디에스티,이매진아시아,SK머티리얼즈
2,0.048872,0.095435,0.064000,-0.050000,-0.057203,-0.053521,0.083478,-0.371274,0.114223,0.077333,...,0.568669,-0.283591,-0.191083,-0.200828,0.091987,-0.138672,-0.241192,-0.081818,-0.226088,-0.625000
3,-0.082437,-0.321970,-0.161654,-0.295322,-0.260674,-0.334821,-0.346709,0.514355,-0.251261,-0.175636,...,0.105355,0.012100,-0.157480,0.880829,0.657701,1.573127,1.410714,0.072278,0.818430,1.056738
4,0.308594,0.374303,-0.044843,0.112033,0.246201,0.118568,0.000000,-0.442941,0.055098,-0.106495,...,0.809253,-0.195559,0.214953,-0.134986,-0.028077,-0.260563,-0.140741,-0.094738,1.091188,-0.255172
5,-0.104478,-0.284554,-0.184977,-0.220149,0.129268,0.008000,0.019656,-0.487802,-0.193044,-0.162362,...,-0.373184,-0.232484,-0.291538,-0.331210,-0.344446,-0.419048,0.405172,-0.267441,-0.371628,-0.203704
6,0.026667,0.000000,-0.020737,0.157895,-0.071274,0.226190,-0.091566,0.535297,-0.137239,0.132159,...,-0.337569,-0.004149,-0.131379,-0.180952,-0.220334,-0.204918,0.779141,-0.142857,0.156411,-0.059302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,-0.067358,-0.092995,-0.046512,0.395257,0.138365,0.050743,0.090024,0.184869,0.117574,0.090379,...,0.000000,-0.078397,0.396433,0.087527,0.000000,0.073460,0.000000,0.000000,0.000000,0.006120
250,0.077778,0.118509,0.062331,0.011331,-0.041989,0.104829,0.026786,-0.104429,-0.103989,-0.026738,...,0.000000,-0.034026,-0.009823,0.006036,0.000000,-0.101545,0.000000,0.000000,0.000000,-0.077453
251,-0.027491,-0.048810,0.061224,-0.078431,0.019608,0.076759,-0.019565,-0.094641,-0.085029,-0.016484,...,0.000000,0.003914,-0.016865,-0.026000,0.000000,-0.068796,0.000000,0.000000,0.000000,-0.023736
252,0.178445,0.220275,0.125000,0.103343,0.205882,0.144554,0.082040,0.115147,0.041297,0.050279,...,0.000000,0.068226,0.015136,0.141684,0.000000,0.125330,0.000000,0.000000,0.000000,0.169293


In [32]:
rStock_data.shape

(252, 2)

In [33]:
data = pd.concat([rStock_data,returns_df], axis=1)

In [34]:
data.reset_index()
data

,date,risk_free,삼성전자,SK하이닉스,POSCO홀딩스,현대차,삼성SDI,기아,현대모비스,카카오,...,행남사,우리종금,부산가스,한국아트라스비엑스,미래SCI,녹십자셀,코너스톤네트웍스,디에스티,이매진아시아,SK머티리얼즈
2,2000-01,0.006667,0.048872,0.095435,0.064000,-0.050000,-0.057203,-0.053521,0.083478,-0.371274,...,0.568669,-0.283591,-0.191083,-0.200828,0.091987,-0.138672,-0.241192,-0.081818,-0.226088,-0.625000
3,2000-02,0.006667,-0.082437,-0.321970,-0.161654,-0.295322,-0.260674,-0.334821,-0.346709,0.514355,...,0.105355,0.012100,-0.157480,0.880829,0.657701,1.573127,1.410714,0.072278,0.818430,1.056738
4,2000-03,0.006667,0.308594,0.374303,-0.044843,0.112033,0.246201,0.118568,0.000000,-0.442941,...,0.809253,-0.195559,0.214953,-0.134986,-0.028077,-0.260563,-0.140741,-0.094738,1.091188,-0.255172
5,2000-04,0.006667,-0.104478,-0.284554,-0.184977,-0.220149,0.129268,0.008000,0.019656,-0.487802,...,-0.373184,-0.232484,-0.291538,-0.331210,-0.344446,-0.419048,0.405172,-0.267441,-0.371628,-0.203704
6,2000-05,0.006667,0.026667,0.000000,-0.020737,0.157895,-0.071274,0.226190,-0.091566,0.535297,...,-0.337569,-0.004149,-0.131379,-0.180952,-0.220334,-0.204918,0.779141,-0.142857,0.156411,-0.059302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,2020-08,0.000821,-0.067358,-0.092995,-0.046512,0.395257,0.138365,0.050743,0.090024,0.184869,...,0.000000,-0.078397,0.396433,0.087527,0.000000,0.073460,0.000000,0.000000,0.000000,0.006120
250,2020-09,0.000821,0.077778,0.118509,0.062331,0.011331,-0.041989,0.104829,0.026786,-0.104429,...,0.000000,-0.034026,-0.009823,0.006036,0.000000,-0.101545,0.000000,0.000000,0.000000,-0.077453
251,2020-10,0.000821,-0.027491,-0.048810,0.061224,-0.078431,0.019608,0.076759,-0.019565,-0.094641,...,0.000000,0.003914,-0.016865,-0.026000,0.000000,-0.068796,0.000000,0.000000,0.000000,-0.023736
252,2020-11,0.000821,0.178445,0.220275,0.125000,0.103343,0.205882,0.144554,0.082040,0.115147,...,0.000000,0.068226,0.015136,0.141684,0.000000,0.125330,0.000000,0.000000,0.000000,0.169293


In [35]:
data.to_csv("rate_dataset.csv", index=None, mode='w')

# 2. Markowitz 계산

In [36]:
from cvxopt import matrix
from cvxopt.solvers import qp
import numpy as np
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch import nn, optim, cuda, Tensor
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import time
import copy

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/MARKET_KOREA/rate_dataset.csv', delimiter=',')

In [39]:
df

,date,risk_free,삼성전자,SK하이닉스,POSCO홀딩스,현대차,삼성SDI,기아,현대모비스,카카오,...,행남사,우리종금,부산가스,한국아트라스비엑스,미래SCI,녹십자셀,코너스톤네트웍스,디에스티,이매진아시아,SK머티리얼즈
0,2000-01,0.006667,0.048872,0.095435,0.064000,-0.050000,-0.057203,-0.053521,0.083478,-0.371274,...,0.568669,-0.283591,-0.191083,-0.200828,0.091987,-0.138672,-0.241192,-0.081818,-0.226088,-0.625000
1,2000-02,0.006667,-0.082437,-0.321970,-0.161654,-0.295322,-0.260674,-0.334821,-0.346709,0.514355,...,0.105355,0.012100,-0.157480,0.880829,0.657701,1.573127,1.410714,0.072278,0.818430,1.056738
2,2000-03,0.006667,0.308594,0.374303,-0.044843,0.112033,0.246201,0.118568,0.000000,-0.442941,...,0.809253,-0.195559,0.214953,-0.134986,-0.028077,-0.260563,-0.140741,-0.094738,1.091188,-0.255172
3,2000-04,0.006667,-0.104478,-0.284554,-0.184977,-0.220149,0.129268,0.008000,0.019656,-0.487802,...,-0.373184,-0.232484,-0.291538,-0.331210,-0.344446,-0.419048,0.405172,-0.267441,-0.371628,-0.203704
4,2000-05,0.006667,0.026667,0.000000,-0.020737,0.157895,-0.071274,0.226190,-0.091566,0.535297,...,-0.337569,-0.004149,-0.131379,-0.180952,-0.220334,-0.204918,0.779141,-0.142857,0.156411,-0.059302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2020-08,0.000821,-0.067358,-0.092995,-0.046512,0.395257,0.138365,0.050743,0.090024,0.184869,...,0.000000,-0.078397,0.396433,0.087527,0.000000,0.073460,0.000000,0.000000,0.000000,0.006120
248,2020-09,0.000821,0.077778,0.118509,0.062331,0.011331,-0.041989,0.104829,0.026786,-0.104429,...,0.000000,-0.034026,-0.009823,0.006036,0.000000,-0.101545,0.000000,0.000000,0.000000,-0.077453
249,2020-10,0.000821,-0.027491,-0.048810,0.061224,-0.078431,0.019608,0.076759,-0.019565,-0.094641,...,0.000000,0.003914,-0.016865,-0.026000,0.000000,-0.068796,0.000000,0.000000,0.000000,-0.023736
250,2020-11,0.000821,0.178445,0.220275,0.125000,0.103343,0.205882,0.144554,0.082040,0.115147,...,0.000000,0.068226,0.015136,0.141684,0.000000,0.125330,0.000000,0.000000,0.000000,0.169293


In [40]:
df.columns

Index(['date', 'risk_free', '삼성전자', 'SK하이닉스', 'POSCO홀딩스', '현대차', '삼성SDI', '기아',
       '현대모비스', '카카오',
       ...
       '행남사', '우리종금', '부산가스', '한국아트라스비엑스', '미래SCI', '녹십자셀', '코너스톤네트웍스', '디에스티',
       '이매진아시아', 'SK머티리얼즈'],
      dtype='object', length=666)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Columns: 666 entries, date to SK머티리얼즈
dtypes: float64(665), object(1)
memory usage: 1.3+ MB


In [42]:
window_size = 120 #12*10 for 10year data to backtest

### test for i=0

In [55]:
df

,date,risk_free,삼성전자,SK하이닉스,POSCO홀딩스,현대차,삼성SDI,기아,현대모비스,카카오,...,행남사,우리종금,부산가스,한국아트라스비엑스,미래SCI,녹십자셀,코너스톤네트웍스,디에스티,이매진아시아,SK머티리얼즈
0,2000-01,0.006667,0.048872,0.095435,0.064000,-0.050000,-0.057203,-0.053521,0.083478,-0.371274,...,0.568669,-0.283591,-0.191083,-0.200828,0.091987,-0.138672,-0.241192,-0.081818,-0.226088,-0.625000
1,2000-02,0.006667,-0.082437,-0.321970,-0.161654,-0.295322,-0.260674,-0.334821,-0.346709,0.514355,...,0.105355,0.012100,-0.157480,0.880829,0.657701,1.573127,1.410714,0.072278,0.818430,1.056738
2,2000-03,0.006667,0.308594,0.374303,-0.044843,0.112033,0.246201,0.118568,0.000000,-0.442941,...,0.809253,-0.195559,0.214953,-0.134986,-0.028077,-0.260563,-0.140741,-0.094738,1.091188,-0.255172
3,2000-04,0.006667,-0.104478,-0.284554,-0.184977,-0.220149,0.129268,0.008000,0.019656,-0.487802,...,-0.373184,-0.232484,-0.291538,-0.331210,-0.344446,-0.419048,0.405172,-0.267441,-0.371628,-0.203704
4,2000-05,0.006667,0.026667,0.000000,-0.020737,0.157895,-0.071274,0.226190,-0.091566,0.535297,...,-0.337569,-0.004149,-0.131379,-0.180952,-0.220334,-0.204918,0.779141,-0.142857,0.156411,-0.059302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2020-08,0.000821,-0.067358,-0.092995,-0.046512,0.395257,0.138365,0.050743,0.090024,0.184869,...,0.000000,-0.078397,0.396433,0.087527,0.000000,0.073460,0.000000,0.000000,0.000000,0.006120
248,2020-09,0.000821,0.077778,0.118509,0.062331,0.011331,-0.041989,0.104829,0.026786,-0.104429,...,0.000000,-0.034026,-0.009823,0.006036,0.000000,-0.101545,0.000000,0.000000,0.000000,-0.077453
249,2020-10,0.000821,-0.027491,-0.048810,0.061224,-0.078431,0.019608,0.076759,-0.019565,-0.094641,...,0.000000,0.003914,-0.016865,-0.026000,0.000000,-0.068796,0.000000,0.000000,0.000000,-0.023736
250,2020-11,0.000821,0.178445,0.220275,0.125000,0.103343,0.205882,0.144554,0.082040,0.115147,...,0.000000,0.068226,0.015136,0.141684,0.000000,0.125330,0.000000,0.000000,0.000000,0.169293


In [ ]:
stock_df = df.drop(['risk_free'],axis=1, inplace=False)

In [51]:
stock_df

,date,삼성전자,SK하이닉스,POSCO홀딩스,현대차,삼성SDI,기아,현대모비스,카카오,LG,...,행남사,우리종금,부산가스,한국아트라스비엑스,미래SCI,녹십자셀,코너스톤네트웍스,디에스티,이매진아시아,SK머티리얼즈
0,2000-01,0.048872,0.095435,0.064000,-0.050000,-0.057203,-0.053521,0.083478,-0.371274,0.114223,...,0.568669,-0.283591,-0.191083,-0.200828,0.091987,-0.138672,-0.241192,-0.081818,-0.226088,-0.625000
1,2000-02,-0.082437,-0.321970,-0.161654,-0.295322,-0.260674,-0.334821,-0.346709,0.514355,-0.251261,...,0.105355,0.012100,-0.157480,0.880829,0.657701,1.573127,1.410714,0.072278,0.818430,1.056738
2,2000-03,0.308594,0.374303,-0.044843,0.112033,0.246201,0.118568,0.000000,-0.442941,0.055098,...,0.809253,-0.195559,0.214953,-0.134986,-0.028077,-0.260563,-0.140741,-0.094738,1.091188,-0.255172
3,2000-04,-0.104478,-0.284554,-0.184977,-0.220149,0.129268,0.008000,0.019656,-0.487802,-0.193044,...,-0.373184,-0.232484,-0.291538,-0.331210,-0.344446,-0.419048,0.405172,-0.267441,-0.371628,-0.203704
4,2000-05,0.026667,0.000000,-0.020737,0.157895,-0.071274,0.226190,-0.091566,0.535297,-0.137239,...,-0.337569,-0.004149,-0.131379,-0.180952,-0.220334,-0.204918,0.779141,-0.142857,0.156411,-0.059302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2020-08,-0.067358,-0.092995,-0.046512,0.395257,0.138365,0.050743,0.090024,0.184869,0.117574,...,0.000000,-0.078397,0.396433,0.087527,0.000000,0.073460,0.000000,0.000000,0.000000,0.006120
248,2020-09,0.077778,0.118509,0.062331,0.011331,-0.041989,0.104829,0.026786,-0.104429,-0.103989,...,0.000000,-0.034026,-0.009823,0.006036,0.000000,-0.101545,0.000000,0.000000,0.000000,-0.077453
249,2020-10,-0.027491,-0.048810,0.061224,-0.078431,0.019608,0.076759,-0.019565,-0.094641,-0.085029,...,0.000000,0.003914,-0.016865,-0.026000,0.000000,-0.068796,0.000000,0.000000,0.000000,-0.023736
250,2020-11,0.178445,0.220275,0.125000,0.103343,0.205882,0.144554,0.082040,0.115147,0.041297,...,0.000000,0.068226,0.015136,0.141684,0.000000,0.125330,0.000000,0.000000,0.000000,0.169293


In [52]:
window = stock_df.iloc[0:0 + window_size,:]
window

,date,삼성전자,SK하이닉스,POSCO홀딩스,현대차,삼성SDI,기아,현대모비스,카카오,LG,...,행남사,우리종금,부산가스,한국아트라스비엑스,미래SCI,녹십자셀,코너스톤네트웍스,디에스티,이매진아시아,SK머티리얼즈
0,2000-01,0.048872,0.095435,0.064000,-0.050000,-0.057203,-0.053521,0.083478,-0.371274,0.114223,...,0.568669,-0.283591,-0.191083,-0.200828,0.091987,-0.138672,-0.241192,-0.081818,-0.226088,-0.625000
1,2000-02,-0.082437,-0.321970,-0.161654,-0.295322,-0.260674,-0.334821,-0.346709,0.514355,-0.251261,...,0.105355,0.012100,-0.157480,0.880829,0.657701,1.573127,1.410714,0.072278,0.818430,1.056738
2,2000-03,0.308594,0.374303,-0.044843,0.112033,0.246201,0.118568,0.000000,-0.442941,0.055098,...,0.809253,-0.195559,0.214953,-0.134986,-0.028077,-0.260563,-0.140741,-0.094738,1.091188,-0.255172
3,2000-04,-0.104478,-0.284554,-0.184977,-0.220149,0.129268,0.008000,0.019656,-0.487802,-0.193044,...,-0.373184,-0.232484,-0.291538,-0.331210,-0.344446,-0.419048,0.405172,-0.267441,-0.371628,-0.203704
4,2000-05,0.026667,0.000000,-0.020737,0.157895,-0.071274,0.226190,-0.091566,0.535297,-0.137239,...,-0.337569,-0.004149,-0.131379,-0.180952,-0.220334,-0.204918,0.779141,-0.142857,0.156411,-0.059302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2009-08,0.064917,0.247159,-0.080677,0.203178,0.450980,0.100977,0.023166,-0.106005,0.149543,...,-0.192985,0.274302,0.052758,0.145270,0.005308,0.240030,-0.107143,-0.052803,-0.272781,0.074401
116,2009-09,0.057069,-0.097950,0.055255,0.051887,0.006757,0.100592,0.249057,0.212612,0.038100,...,-0.197571,0.121000,0.034169,0.047198,-0.026220,-0.230784,-0.010000,-0.055460,-0.076824,0.124413
117,2009-10,-0.112883,-0.088384,0.034908,-0.017937,-0.077181,-0.032258,-0.030211,0.016572,-0.134172,...,0.066225,-0.135706,-0.011013,-0.061972,0.032349,0.016127,0.262626,0.127421,-0.062456,-0.123173
118,2009-11,-0.004149,0.022161,0.107143,-0.095890,-0.080000,-0.047222,-0.071651,0.137891,-0.070169,...,-0.141970,-0.065158,-0.020045,-0.156156,0.001811,-0.358713,-0.004000,-0.052081,0.333271,0.046429


In [56]:
#date 빼기
n=window.shape[1]-1
#risk free 0는 i로 변경(2010년도)
r_min = df.iloc[0+window_size+1,1]

In [57]:
r_min

0.003048368

In [59]:
#cov
Cov = window.cov()
Cov

<ipython-input-59-a1150d120896>:2: FutureWarning: The default value of numeric_only in DataFrame.cov is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Cov = window.cov()


,삼성전자,SK하이닉스,POSCO홀딩스,현대차,삼성SDI,기아,현대모비스,카카오,LG,삼성화재,...,행남사,우리종금,부산가스,한국아트라스비엑스,미래SCI,녹십자셀,코너스톤네트웍스,디에스티,이매진아시아,SK머티리얼즈
삼성전자,0.012797,0.017874,0.004608,0.008790,0.007838,0.006549,0.007585,0.009003,0.008062,0.005139,...,0.004836,0.008380,0.000276,0.002664,0.012783,0.006580,0.005787,0.004334,0.010129,0.007259
SK하이닉스,0.017874,0.068007,0.007752,0.017110,0.012638,0.013944,0.011825,0.018360,0.016496,0.011555,...,0.008681,0.019786,0.004361,0.004791,0.024974,0.020304,0.009858,0.013998,0.025685,0.011643
POSCO홀딩스,0.004608,0.007752,0.008958,0.004925,0.003104,0.003867,0.005859,0.007638,0.005915,0.004020,...,0.002774,0.006786,0.001544,0.002983,0.007616,0.002774,0.002398,0.003377,0.004824,0.004301
현대차,0.008790,0.017110,0.004925,0.015129,0.010708,0.012389,0.012007,0.007894,0.006869,0.006395,...,0.003851,0.009987,0.002959,0.004240,0.009448,0.011170,0.003905,0.004718,0.009040,0.006290
삼성SDI,0.007838,0.012638,0.003104,0.010708,0.019745,0.010227,0.010262,0.005987,0.009273,0.005409,...,0.003331,0.005803,0.001771,0.002160,0.010528,0.009021,0.004000,0.004067,0.008175,0.008049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
녹십자셀,0.006580,0.020304,0.002774,0.011170,0.009021,0.013914,0.007133,0.026082,0.012935,0.010552,...,0.016613,0.025681,0.002967,0.029520,0.036333,0.497623,0.064778,0.006796,0.017173,0.028934
코너스톤네트웍스,0.005787,0.009858,0.002398,0.003905,0.004000,0.002697,0.002293,0.034023,0.004520,0.004783,...,-0.000268,0.010223,-0.003976,0.020534,0.040160,0.064778,0.074605,0.009220,0.046749,0.029315
디에스티,0.004334,0.013998,0.003377,0.004718,0.004067,0.001315,0.008947,0.010199,0.008386,0.003634,...,0.001436,0.006909,0.005661,0.007221,0.006394,0.006796,0.009220,0.106748,0.006342,0.008860
이매진아시아,0.010129,0.025685,0.004824,0.009040,0.008175,0.006831,0.001532,0.032715,0.009363,0.008994,...,0.018311,0.015507,0.004570,0.003511,0.053949,0.017173,0.046749,0.006342,0.170826,0.022991


In [60]:
Cov=Cov.to_numpy()
Cov = matrix(Cov)

In [61]:
Mean = window.mean().to_numpy()
Mean
Mean = matrix(Mean)
Mean

<ipython-input-61-4e24cf51a148>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  Mean = window.mean().to_numpy()


<664x1 matrix, tc='d'>

In [62]:
n

664

In [63]:
G = matrix(np.concatenate((-np.transpose(Mean), -np.identity(n)), 0))
h = matrix(np.concatenate((-np.ones((1,1))*r_min, np.zeros((n,1))), 0))

In [64]:
A = matrix(1.0, (1,n))
b = matrix(1.0)
q = matrix(np.zeros((n, 1)))
sol = qp(Cov, q, G, h, A, b)
sol['x']

     pcost       dcost       gap    pres   dres
 0:  1.7988e-03 -1.0160e+00  7e+02  3e+01  3e+01
 1:  6.4548e-03 -1.0173e+00  1e+01  3e-01  3e-01
 2:  4.2136e-03 -8.1244e-01  8e-01  3e-16  5e-15
 3:  4.1830e-03 -1.1929e-02  2e-02  1e-15  9e-15
 4:  3.5289e-03 -3.2202e-03  7e-03  9e-16  3e-15
 5:  2.7725e-03 -4.1453e-03  7e-03  8e-16  3e-15
 6:  1.7400e-03 -7.6948e-04  3e-03  4e-16  7e-16
 7:  1.7048e-03 -1.5993e-03  3e-03  2e-16  5e-16
 8:  1.0148e-03  1.0203e-04  9e-04  3e-16  7e-17
 9:  8.0416e-04  3.4858e-04  5e-04  4e-16  4e-17
10:  6.7252e-04  4.8080e-04  2e-04  7e-16  3e-17
11:  5.8488e-04  5.3087e-04  5e-05  2e-16  3e-17
12:  5.4998e-04  5.4232e-04  8e-06  7e-16  3e-17
13:  5.4342e-04  5.4311e-04  3e-07  2e-16  3e-17
14:  5.4314e-04  5.4313e-04  4e-09  4e-16  3e-17
Optimal solution found.


<664x1 matrix, tc='d'>

In [65]:
w = np.asarray(np.squeeze(np.asarray(sol['x'])))

In [66]:
w

array([3.38233797e-08, 2.27350378e-09, 1.14418690e-08, 1.13253705e-08,
       1.27963781e-08, 8.86552952e-09, 6.43976965e-09, 3.49951837e-09,
       4.49011702e-09, 2.00328450e-08, 2.56847352e-01, 3.72715025e-07,
       6.45916663e-02, 4.55002894e-09, 7.11830748e-09, 4.56492364e-09,
       1.46800495e-08, 4.76337177e-03, 7.22578275e-09, 4.21151368e-09,
       1.99580781e-09, 4.41094475e-09, 6.38483474e-09, 2.79940336e-09,
       3.07697940e-09, 5.37096795e-09, 6.99499854e-09, 6.09231810e-09,
       4.12911811e-09, 2.93297174e-09, 7.22913352e-09, 1.50505927e-08,
       4.45190114e-09, 2.78315744e-09, 3.70331829e-09, 3.25659283e-09,
       2.55594935e-09, 5.03889787e-09, 5.00133753e-09, 6.20561580e-09,
       7.32749997e-09, 4.93866262e-09, 6.70706830e-09, 2.45812426e-09,
       4.78438055e-09, 2.72202198e-09, 1.53042134e-02, 5.14034660e-02,
       6.90898165e-09, 1.34748061e-02, 5.84625476e-09, 3.11504657e-08,
       5.66644578e-09, 4.62143538e-09, 8.02900210e-06, 7.75813324e-09,
      

In [67]:
print(w.shape)

(664,)


In [68]:
df.iloc[0+window_size][:]

date          2010-01
risk_free    0.003048
삼성전자        -0.018773
SK하이닉스      -0.017279
POSCO홀딩스    -0.132686
               ...   
녹십자셀         0.043887
코너스톤네트웍스     0.107143
디에스티              0.0
이매진아시아      -0.168908
SK머티리얼즈     -0.123548
Name: 120, Length: 666, dtype: object

## Now test for whole backtest code

In [ ]:
window_size = 120 #12*10 for 10year data to backtest

In [71]:
#result is from 2010~2020
# Initialize an new dataframe for makwowitz weight(so except risk free)
mk_df = pd.DataFrame(df.iloc[120:][:])
#result of protfolio(so use mk_df and date for r)
result = pd.DataFrame()

In [72]:
result['date'] = df.iloc[120:]['date']
result['r'] = None
result.reset_index(drop=True, inplace=True)

In [73]:
result.shape

(132, 2)

In [74]:
result.loc[0, 'r'] = 1
result.head()

,date,r
0,2010-01,1
1,2010-02,None
2,2010-03,None
3,2010-04,None
4,2010-05,None


In [75]:
mk_df.drop(['risk_free'], axis=1, inplace= True)
mk_df.reset_index(drop=True, inplace=True)

In [76]:
mk_df.iloc[:,1:] = 0
mk_df

<ipython-input-76-3cd3589eb226>:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  mk_df.iloc[:,1:] = 0


,date,삼성전자,SK하이닉스,POSCO홀딩스,현대차,삼성SDI,기아,현대모비스,카카오,LG,...,행남사,우리종금,부산가스,한국아트라스비엑스,미래SCI,녹십자셀,코너스톤네트웍스,디에스티,이매진아시아,SK머티리얼즈
0,2010-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2010-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2010-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2010-04,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2010-05,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,2020-08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
128,2020-09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
129,2020-10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
130,2020-11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
len(df)

252

In [78]:
stock_df = df.drop(['risk_free'],axis=1, inplace=False)

In [80]:
stock_df

,date,삼성전자,SK하이닉스,POSCO홀딩스,현대차,삼성SDI,기아,현대모비스,카카오,LG,...,행남사,우리종금,부산가스,한국아트라스비엑스,미래SCI,녹십자셀,코너스톤네트웍스,디에스티,이매진아시아,SK머티리얼즈
0,2000-01,0.048872,0.095435,0.064000,-0.050000,-0.057203,-0.053521,0.083478,-0.371274,0.114223,...,0.568669,-0.283591,-0.191083,-0.200828,0.091987,-0.138672,-0.241192,-0.081818,-0.226088,-0.625000
1,2000-02,-0.082437,-0.321970,-0.161654,-0.295322,-0.260674,-0.334821,-0.346709,0.514355,-0.251261,...,0.105355,0.012100,-0.157480,0.880829,0.657701,1.573127,1.410714,0.072278,0.818430,1.056738
2,2000-03,0.308594,0.374303,-0.044843,0.112033,0.246201,0.118568,0.000000,-0.442941,0.055098,...,0.809253,-0.195559,0.214953,-0.134986,-0.028077,-0.260563,-0.140741,-0.094738,1.091188,-0.255172
3,2000-04,-0.104478,-0.284554,-0.184977,-0.220149,0.129268,0.008000,0.019656,-0.487802,-0.193044,...,-0.373184,-0.232484,-0.291538,-0.331210,-0.344446,-0.419048,0.405172,-0.267441,-0.371628,-0.203704
4,2000-05,0.026667,0.000000,-0.020737,0.157895,-0.071274,0.226190,-0.091566,0.535297,-0.137239,...,-0.337569,-0.004149,-0.131379,-0.180952,-0.220334,-0.204918,0.779141,-0.142857,0.156411,-0.059302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2020-08,-0.067358,-0.092995,-0.046512,0.395257,0.138365,0.050743,0.090024,0.184869,0.117574,...,0.000000,-0.078397,0.396433,0.087527,0.000000,0.073460,0.000000,0.000000,0.000000,0.006120
248,2020-09,0.077778,0.118509,0.062331,0.011331,-0.041989,0.104829,0.026786,-0.104429,-0.103989,...,0.000000,-0.034026,-0.009823,0.006036,0.000000,-0.101545,0.000000,0.000000,0.000000,-0.077453
249,2020-10,-0.027491,-0.048810,0.061224,-0.078431,0.019608,0.076759,-0.019565,-0.094641,-0.085029,...,0.000000,0.003914,-0.016865,-0.026000,0.000000,-0.068796,0.000000,0.000000,0.000000,-0.023736
250,2020-11,0.178445,0.220275,0.125000,0.103343,0.205882,0.144554,0.082040,0.115147,0.041297,...,0.000000,0.068226,0.015136,0.141684,0.000000,0.125330,0.000000,0.000000,0.000000,0.169293


widow size = 120

252-120+1=133(132+1)

In [82]:
#code ver2
# Iterate through the dataframe using a sliding window
for i in range(len(df) - window_size):
    #sliding window for i
    window = stock_df.iloc[i:i + window_size, :]
    # Perform computations on the window
    n=window.shape[1]-1
    r_min = df.iloc[0+window_size+1][1]
    Cov = matrix(window.cov().to_numpy())
    Mean = matrix(window.mean().to_numpy())
    A = matrix(1.0, (1,n))
    b = matrix(1.0)
    q = matrix(np.zeros((n, 1)))
    sol = qp(Cov, q, G, h, A, b)
    w = np.asarray(np.squeeze(np.asarray(sol['x'])))
    mk_df.iloc[i, 1:] = w
    result.loc[i, 'r'] = w.dot(df.iloc[i+window_size][2:].transpose())

<ipython-input-82-a24ad428c791>:9: FutureWarning: The default value of numeric_only in DataFrame.cov is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Cov = matrix(window.cov().to_numpy())
<ipython-input-82-a24ad428c791>:10: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  Mean = matrix(window.mean().to_numpy())


     pcost       dcost       gap    pres   dres
 0:  1.7988e-03 -1.0160e+00  7e+02  3e+01  3e+01
 1:  6.4548e-03 -1.0173e+00  1e+01  3e-01  3e-01
 2:  4.2136e-03 -8.1244e-01  8e-01  3e-16  5e-15
 3:  4.1830e-03 -1.1929e-02  2e-02  1e-15  9e-15
 4:  3.5289e-03 -3.2202e-03  7e-03  9e-16  3e-15
 5:  2.7725e-03 -4.1453e-03  7e-03  8e-16  3e-15
 6:  1.7400e-03 -7.6948e-04  3e-03  4e-16  7e-16
 7:  1.7048e-03 -1.5993e-03  3e-03  2e-16  5e-16
 8:  1.0148e-03  1.0203e-04  9e-04  3e-16  7e-17
 9:  8.0416e-04  3.4858e-04  5e-04  4e-16  4e-17
10:  6.7252e-04  4.8080e-04  2e-04  7e-16  3e-17
11:  5.8488e-04  5.3087e-04  5e-05  2e-16  3e-17
12:  5.4998e-04  5.4232e-04  8e-06  7e-16  3e-17
13:  5.4342e-04  5.4311e-04  3e-07  2e-16  3e-17
14:  5.4314e-04  5.4313e-04  4e-09  4e-16  3e-17
Optimal solution found.


<ipython-input-82-a24ad428c791>:9: FutureWarning: The default value of numeric_only in DataFrame.cov is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Cov = matrix(window.cov().to_numpy())
<ipython-input-82-a24ad428c791>:10: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  Mean = matrix(window.mean().to_numpy())


     pcost       dcost       gap    pres   dres
 0:  1.8005e-03 -1.0156e+00  7e+02  3e+01  3e+01
 1:  6.5061e-03 -1.0171e+00  1e+01  3e-01  3e-01
 2:  4.2170e-03 -8.1205e-01  8e-01  2e-16  5e-15
 3:  4.1865e-03 -1.1958e-02  2e-02  4e-16  8e-15
 4:  3.5357e-03 -3.2541e-03  7e-03  9e-16  3e-15
 5:  2.7866e-03 -4.1661e-03  7e-03  9e-16  3e-15
 6:  1.7402e-03 -8.0979e-04  3e-03  6e-17  7e-16
 7:  1.7141e-03 -1.6488e-03  3e-03  6e-17  5e-16
 8:  1.0396e-03  3.8318e-05  1e-03  4e-16  8e-17
 9:  8.2197e-04  3.1684e-04  5e-04  7e-16  4e-17
10:  6.9401e-04  4.4578e-04  2e-04  1e-16  3e-17
11:  5.9055e-04  5.2273e-04  7e-05  4e-16  3e-17
12:  5.5064e-04  5.3732e-04  1e-05  1e-16  3e-17
13:  5.4181e-04  5.3886e-04  3e-06  2e-16  3e-17
14:  5.3912e-04  5.3907e-04  5e-08  4e-16  3e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.8025e-03 -1.0160e+00  7e+02  3e+01  3e+01


<ipython-input-82-a24ad428c791>:9: FutureWarning: The default value of numeric_only in DataFrame.cov is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Cov = matrix(window.cov().to_numpy())
<ipython-input-82-a24ad428c791>:10: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  Mean = matrix(window.mean().to_numpy())


...

     pcost       dcost       gap    pres   dres
 0:  8.9623e-04 -1.0240e+00  7e+02  3e+01  3e+01
 1:  1.5381e-03 -1.0215e+00  9e+00  3e-01  3e-01
 2:  1.4080e-03 -7.9317e-01  8e-01  2e-16  5e-15
 3:  1.4051e-03 -1.1065e-02  1e-02  3e-16  9e-15
 4:  1.2813e-03 -2.8387e-03  4e-03  3e-16  3e-15
 5:  8.9098e-04 -2.0422e-03  3e-03  1e-16  4e-17
 6:  6.3542e-04 -3.1962e-04  1e-03  7e-16  4e-17
 7:  3.3966e-04 -1.7258e-04  5e-04  1e-16  2e-17
 8:  2.4348e-04  1.1050e-04  1e-04  4e-16  2e-17
 9:  1.9835e-04  1.5373e-04  4e-05  1e-15  1e-17
10:  1.7628e-04  1.6599e-04  1e-05  2e-16  1e-17
11:  1.7103e-04  1.6805e-04  3e-06  7e-16  1e-17
12:  1.6854e-04  1.6833e-04  2e-07  7e-16  1e-17
13:  1.6835e-04  1.6835e-04  3e-09  2e-16  1e-17
Optimal solution found.


<ipython-input-82-a24ad428c791>:9: FutureWarning: The default value of numeric_only in DataFrame.cov is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Cov = matrix(window.cov().to_numpy())
<ipython-input-82-a24ad428c791>:10: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  Mean = matrix(window.mean().to_numpy())


     pcost       dcost       gap    pres   dres
 0:  8.8984e-04 -1.0241e+00  7e+02  3e+01  3e+01
 1:  1.5149e-03 -1.0216e+00  9e+00  3e-01  3e-01
 2:  1.3954e-03 -7.9181e-01  8e-01  7e-16  5e-15
 3:  1.3925e-03 -1.0973e-02  1e-02  2e-16  9e-15
 4:  1.2689e-03 -2.7673e-03  4e-03  2e-16  3e-15
 5:  8.8019e-04 -1.9655e-03  3e-03  2e-16  4e-17
 6:  6.2761e-04 -2.8373e-04  9e-04  4e-16  4e-17
 7:  3.8338e-04 -2.4041e-04  6e-04  7e-16  2e-17
 8:  2.6348e-04  8.3716e-05  2e-04  7e-16  2e-17
 9:  2.0574e-04  1.4945e-04  6e-05  7e-16  1e-17
10:  1.7751e-04  1.6644e-04  1e-05  4e-17  1e-17
11:  1.7152e-04  1.6798e-04  4e-06  1e-16  1e-17
12:  1.6851e-04  1.6832e-04  2e-07  1e-16  1e-17
13:  1.6833e-04  1.6833e-04  2e-09  2e-16  1e-17
Optimal solution found.


<ipython-input-82-a24ad428c791>:9: FutureWarning: The default value of numeric_only in DataFrame.cov is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Cov = matrix(window.cov().to_numpy())
<ipython-input-82-a24ad428c791>:10: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  Mean = matrix(window.mean().to_numpy())


     pcost       dcost       gap    pres   dres
 0:  9.2031e-04 -1.0241e+00  7e+02  3e+01  3e+01
 1:  1.5391e-03 -1.0216e+00  9e+00  3e-01  3e-01
 2:  1.4354e-03 -7.8913e-01  8e-01  3e-16  5e-15
 3:  1.4325e-03 -1.1121e-02  1e-02  2e-16  9e-15
 4:  1.3086e-03 -2.9192e-03  4e-03  8e-18  3e-15
 5:  9.2178e-04 -2.0540e-03  3e-03  8e-18  4e-17
 6:  6.8855e-04 -3.5363e-04  1e-03  3e-16  4e-17
 7:  3.9234e-04 -2.1828e-04  6e-04  1e-15  2e-17
 8:  2.6742e-04  1.0897e-04  2e-04  3e-16  2e-17
 9:  2.1554e-04  1.5962e-04  6e-05  2e-16  1e-17
10:  1.9262e-04  1.7220e-04  2e-05  6e-16  1e-17
11:  1.7993e-04  1.7687e-04  3e-06  4e-16  1e-17
12:  1.7742e-04  1.7718e-04  2e-07  1e-16  1e-17
13:  1.7721e-04  1.7720e-04  4e-09  1e-16  1e-17
Optimal solution found.


In [83]:
result

,date,r
0,2010-01,0.002413
1,2010-02,-0.005361
2,2010-03,0.086447
3,2010-04,-0.021287
4,2010-05,0.032469
...,...,...
127,2020-08,0.104815
128,2020-09,0.031689
129,2020-10,0.005171
130,2020-11,0.069201


In [84]:
mk_df

,date,삼성전자,SK하이닉스,POSCO홀딩스,현대차,삼성SDI,기아,현대모비스,카카오,LG,...,행남사,우리종금,부산가스,한국아트라스비엑스,미래SCI,녹십자셀,코너스톤네트웍스,디에스티,이매진아시아,SK머티리얼즈
0,2010-01,3.382338e-08,2.273504e-09,1.144187e-08,1.132537e-08,1.279638e-08,8.865530e-09,6.439770e-09,3.499518e-09,4.490117e-09,...,1.165246e-07,3.528573e-09,1.209133e-08,5.072816e-09,2.853790e-09,1.621720e-09,7.810212e-09,3.458905e-09,8.950191e-09,2.664240e-09
1,2010-02,5.845386e-07,3.623271e-08,1.692395e-07,1.921935e-07,2.055402e-07,1.487171e-07,9.330437e-08,5.732111e-08,6.919187e-08,...,3.242507e-07,5.675612e-08,2.050442e-07,7.700349e-08,4.514708e-08,2.565637e-08,1.275603e-07,5.621079e-08,1.264805e-07,4.510969e-08
2,2010-03,6.247075e-07,2.859240e-08,1.137348e-07,1.353814e-07,1.364127e-07,1.012139e-07,6.539237e-08,5.099806e-08,4.838683e-08,...,5.099407e-07,5.240069e-08,1.436114e-07,7.035889e-08,4.284463e-08,2.435521e-08,2.158582e-07,4.586304e-08,2.354108e-07,4.023902e-08
3,2010-04,2.816131e-08,1.448366e-09,6.628058e-09,7.956569e-09,7.389670e-09,5.020195e-09,3.798806e-09,3.076201e-09,2.744464e-09,...,3.254512e-08,2.557056e-09,8.777235e-09,3.767942e-09,2.811309e-09,1.308903e-09,1.033625e-07,2.386322e-09,1.638353e-08,2.392501e-09
4,2010-05,2.868392e-07,3.132679e-08,1.707001e-07,1.627167e-07,1.094945e-07,9.132209e-08,8.015981e-08,6.985630e-08,6.378925e-08,...,3.537362e-06,5.270002e-08,3.129561e-07,1.073229e-07,6.702634e-08,3.529678e-08,1.609895e-07,6.331237e-08,9.592983e-08,4.857908e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,2020-08,2.356087e-03,4.670388e-02,3.759679e-08,6.762339e-07,1.227661e-07,3.503010e-08,5.712016e-08,1.279882e-07,8.731022e-08,...,2.373002e-08,1.363823e-08,3.930344e-07,3.492746e-08,3.539696e-08,2.303485e-08,1.824148e-08,5.906979e-03,1.358554e-08,2.996070e-08
128,2020-09,1.299032e-02,5.350779e-02,3.328159e-07,3.105150e-07,3.147278e-07,1.936021e-07,2.824636e-07,6.635060e-07,3.343866e-07,...,1.309916e-07,7.517661e-08,4.357642e-07,1.676790e-07,2.954053e-07,1.057640e-07,8.393623e-08,4.151451e-03,7.942241e-08,1.472599e-07
129,2020-10,9.005642e-03,5.249269e-02,3.414115e-08,2.741155e-08,2.928513e-08,1.909739e-08,3.184360e-08,7.939901e-08,3.086719e-08,...,1.411600e-08,7.326048e-09,4.637325e-08,1.590212e-08,1.948912e-08,1.033995e-08,9.062721e-09,3.990423e-03,8.458960e-09,1.429755e-08
130,2020-11,8.188943e-03,5.245501e-02,2.706737e-08,2.320357e-08,2.377288e-08,1.625850e-08,2.633380e-08,6.984445e-08,2.513502e-08,...,1.255528e-08,5.820206e-09,3.845500e-08,1.276498e-08,1.794597e-08,8.189747e-09,7.702004e-09,3.925685e-03,7.502601e-09,1.144297e-08


In [85]:
result.to_csv("result_markowitz.csv", index=None, mode='w')